In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

#cargamos los datos
df = pd.read_csv('credit_score_data.csv')
df.head()

,client_id,age,sex,zone,education,civil_status,children,work_disability,monthly_txs,main_os,...,monthly_outcome,cc_limit,cc_debt,savings_amount,checking_balance,late_payments_last_year,fixed_investment_amount,cash_installment_debt,visa_installment_debt,default_in_next_12_months
0,67388007,33,hombre,2,universidad,soltero,3,no,17,android,...,16748,24619,8414,117713,32621,0,90676,5155,10924,0
1,12550414,21,mujer,9,colegio,soltero,0,no,17,android,...,23766,52583,11252,107455,42529,1,8292,1701,12652,0
2,35965065,27,mujer,13,universidad,soltero,2,no,5,android,...,24277,60781,19782,123253,30244,2,219788,19646,21205,0
3,11598907,45,mujer,6,universidad,casado,3,no,22,ios,...,33020,29361,14243,129994,23370,0,44210,7380,7588,0
4,79587204,45,hombre,2,postgrado,casado,1,no,21,ios,...,23523,53960,12679,240797,35270,0,78286,7308,19002,0


In [6]:
#identificamos las variables categóricas y numéricas
#y separamos la variable objetivo
cat_vars = df.select_dtypes(include=['object']) .columns.tolist()
binary_vars = [c for c in cat_vars if df[c].nunique() == 2]
#aqui separamos las variables categóricas con más de 2 categorías
multi_vars = [c for c in cat_vars if df[c].nunique() > 2]
numeric_features = df.select_dtypes(include=[np.number]).columns.drop('default_in_next_12_months').tolist()
#ahora se separa la variable objetivo
#y se eliminan las variables categóricas del dataframe

print('Variables binarias:', binary_vars)
print('Variables con >2 categorías:', multi_vars)
print('Características numéricas:', numeric_features)

Variables binarias: ['sex', 'work_disability', 'main_os']
Variables con >2 categorías: ['education', 'civil_status']
Características numéricas: ['client_id', 'age', 'zone', 'children', 'monthly_txs', 'devices', 'daily_balance_checks', 'account_quantity', 'cc_quantity', 'cc_limit_usage_avg', 'monthly_income', 'monthly_outcome', 'cc_limit', 'cc_debt', 'savings_amount', 'checking_balance', 'late_payments_last_year', 'fixed_investment_amount', 'cash_installment_debt', 'visa_installment_debt']


In [7]:
#esta es la funcion que se usa para el calculo del ks
def ks_stat(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    return float(max(tpr - fpr))

#label Encoding para variables binarias
df_le = df.copy()
for var in binary_vars:
    df_le[var] = LabelEncoder().fit_transform(df_le[var])

#datos para modelo baseline
y = df_le['default_in_next_12_months']
X_num = df_le[numeric_features]
X_train_num, X_test_num, y_train, y_test = train_test_split(
    X_num, y, test_size=0.3, random_state=42
)
model_num = LogisticRegression(max_iter=1000)
model_num.fit(X_train_num, y_train)
y_pred_num = model_num.predict_proba(X_test_num)[:, 1]
ks_num = ks_stat(y_test, y_pred_num)
print(f'Baseline KS: {ks_num:.4f}')

Baseline KS: 0.7790


In [8]:
#one-hot encoding para variables con +2 categorías
X_all = df_le.drop('default_in_next_12_months', axis=1)
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', sparse_output=False), multi_vars)
], remainder='passthrough')
#se crea el pipeline para el modelo
pipeline = Pipeline([
    ('preprocessor', ct),
    ('model', LogisticRegression(max_iter=1000))
])

#se ejecuta el split y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.3, random_state=30
)
pipeline.fit(X_train, y_train)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
ks_new = ks_stat(y_test, y_pred_proba)
roc_auc_new = roc_auc_score(y_test, y_pred_proba)
#se imprime el resultado
print(f'New Model KS: {ks_new:.4f}')
print(f'New Model ROC AUC: {roc_auc_new:.4f}')

New Model KS: 0.7739
New Model ROC AUC: 0.9545


# Comparación de Resultados
- **KS Baseline:** 0.7739
- **KS Nuevo Modelo:** 0.9545

El modelo con tratamiento de variables categóricas mejoró el estadístico KS en comparación con el baseline. Esto tiene sentido considerando el contexto. En adhición con esto, tambien el one hot encoding permitió ajustar coeficientes específicos para cada categoría, mejorando mucho tambien la separación entre buenos y malos pagadores.